In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#miRNA (HiSeq, miRgene level)
file_name_list = [
          "drive/MyDrive/5703/TCGA-LUAD/Human__TCGA_LUAD__BDGSC__miRNASeq__HS_miR__01_28_2016__BI__Gene__Firehose_RPM_log2.cct",
          ]

id = 0
df = pd.DataFrame({})
for file_name in file_name_list:
    print(file_name)
    data_dict1 = {}
    # Read the file and populate the data dictionary
    with open(str(file_name), 'r') as file:
        for line in file:
            parts = line.strip().split()
            attribute_name = parts[0]
            attribute_values = parts[1:]
            data_dict1[attribute_name] = attribute_values

    # Create a DataFrame from the data dictionary

    if id == 0:
        df = pd.DataFrame(data_dict1)

    else:
        df2 = pd.DataFrame(data_dict1)
        df = df.merge(df2, on='attrib_name', how='inner')

    id += 1
#df = df[df.iloc[:, 1:].isin(['NA']).any(axis=1) == False]
df.shape

drive/MyDrive/5703/TCGA-LUAD/Human__TCGA_LUAD__BDGSC__miRNASeq__HS_miR__01_28_2016__BI__Gene__Firehose_RPM_log2.cct


(450, 810)

In [ ]:
df = df[df.iloc[:, 1:].isin(['NA']).any(axis=1) == False]
df

,attrib_name,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,...,hsa-mir-940,hsa-mir-941-1,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
0,TCGA.05.4384,13.8766,14.8745,13.8822,13.8259,10.6177,8.7119,10.8698,5.3122,15.1357,...,1.0159,0,2.5,0,2.4707,2.0238,3.3015,6.1628,8.3201,14.7985
1,TCGA.05.4390,11.7425,12.7576,11.7578,13.0601,7.608,8.6168,10.4833,3.4069,12.4367,...,2.6638,0,2.5369,0.617,0.4392,4.1762,4.5819,6.9962,5.0913,16.1543
2,TCGA.05.4396,14.0194,15.0255,14.0367,14.5902,11.1171,9.8454,11.4738,4.3995,14.3723,...,1.2395,0.1437,2.4873,0.1437,0.6074,2.1891,3.8691,6.6731,8.0122,13.9981
3,TCGA.05.4405,12.9428,13.9327,12.9499,14.217,11.1093,8.4836,10.3909,3.1985,12.5092,...,1.8467,0,2.1296,0,2.0413,1.1168,4.9047,5.3844,8.5943,14.2003
4,TCGA.05.4410,12.715,13.7157,12.7252,13.7465,10.3613,8.736,10.0696,3.9421,13.0051,...,1.2397,0,3.0982,0,1.3293,0.6827,4.1774,5.0606,8.3316,13.8615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,TCGA.NJ.A55O,13.726,14.7255,13.7416,13.8372,10.6305,9.3335,11.1954,6.08,14.5731,...,0.5203,0,2.1615,0,1.2033,3.6446,3.8967,6.0614,9.0105,13.9573
446,TCGA.NJ.A55R,12.3826,13.3917,12.3986,12.8563,9.2932,8.5693,10.4809,4.2564,12.6567,...,1.3273,0,1.9135,0,0.588,3.8663,6.271,5.5402,7.9113,14.3198
447,TCGA.NJ.A7XG,12.6324,13.6234,12.6361,13.4904,10.1604,7.7891,10.096,4.4593,12.5202,...,0.4228,0,1.169,0.1551,2.787,0.9321,4.4367,4.7605,8.3495,13.9729
448,TCGA.O1.A52J,11.9579,12.9604,11.9678,12.5495,10.7978,9.0521,9.9905,4.819,12.9217,...,1.7695,0,2.4797,0,0.9736,1.4258,4.1976,7.3373,8.9637,14.447


In [ ]:
#row = df[df['attrib_name'] == 'TCGA.A2.A04P']

# Print the values of columns "C5orf40_y" and "C5orf40_x" for the selected row
#print("C5orf40_y:", row['C5orf40_y'].values[0])
#print("C5orf40_x:", row['C5orf40_x'].values[0])

In [ ]:
#Separate text descriptions from numeric attributes
text_descriptions = df.iloc[:,0]
numeric_attributes = df.iloc[:,1:]
print(numeric_attributes)
print(numeric_attributes.shape[1])

    hsa-let-7a-1 hsa-let-7a-2 hsa-let-7a-3 hsa-let-7b hsa-let-7c hsa-let-7d  \
0        13.8766      14.8745      13.8822    13.8259    10.6177     8.7119   
1        11.7425      12.7576      11.7578    13.0601      7.608     8.6168   
2        14.0194      15.0255      14.0367    14.5902    11.1171     9.8454   
3        12.9428      13.9327      12.9499     14.217    11.1093     8.4836   
4         12.715      13.7157      12.7252    13.7465    10.3613      8.736   
..           ...          ...          ...        ...        ...        ...   
445       13.726      14.7255      13.7416    13.8372    10.6305     9.3335   
446      12.3826      13.3917      12.3986    12.8563     9.2932     8.5693   
447      12.6324      13.6234      12.6361    13.4904    10.1604     7.7891   
448      11.9579      12.9604      11.9678    12.5495    10.7978     9.0521   
449      13.2691      14.2701      13.2772     14.037    10.9685     8.3676   

    hsa-let-7e hsa-let-7f-1 hsa-let-7f-2 hsa-let-7g

In [ ]:
pip install keras-tuner --upgrade


In [ ]:
#https://www.analyticsvidhya.com/blog/2021/06/dimensionality-reduction-using-autoencoders-in-python/
import math
import pandas as pd
import tensorflow as tf
import kerastuner.tuners as kt
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import numpy as np


# Train the autoencoder with the training data.
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),#
          Dense(256, activation="relu"),
          Dense(128, activation="relu"),

        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(512, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded


def create_and_train_autoencoder(x_train_scaled,x_test_scaled):
    auto_encoder = AutoEncoders(len(x_train_scaled.columns))
    auto_encoder.compile(
        loss='mae',
        metrics=['mae'],
        #optimizer='adam'
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)# 0.001 to avoid overlearning
    )
    history = auto_encoder.fit(
        x_train_scaled,
        x_train_scaled,
        epochs=50,
        batch_size=32,
        validation_data=(x_test_scaled, x_test_scaled),
        verbose=0
    )
    return history




In [ ]:
# Split the data
standard_scaler = MinMaxScaler()
numeric_attributes_scaled = pd.DataFrame(standard_scaler.fit_transform(numeric_attributes),columns=numeric_attributes.columns)

In [ ]:
x_train_scaled, x_test_scaled = train_test_split(numeric_attributes_scaled, test_size=0.3, random_state=42)

auto_encoder = AutoEncoders(len(x_train_scaled.columns))
auto_encoder.compile(
    loss='mae',
    metrics=['mae'],
    #optimizer='adam'
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)# 0.001 to avoid overlearning
)
history = auto_encoder.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=50,
    batch_size=32,
    validation_data=(x_test_scaled, x_test_scaled),
    #verbose=0
)
encoder_layer = auto_encoder.layers[0]  # Get the encoder layer
X_encoded = pd.DataFrame(encoder_layer.predict(numeric_attributes_scaled))  # Reduced_df
#X_encoded

Epoch 1/50
10/10 [==============================] - 1s 30ms/step - loss: 0.2310 - mae: 0.2310 - val_loss: 0.1247 - val_mae: 0.1247
Epoch 2/50
10/10 [==============================] - 0s 15ms/step - loss: 0.1145 - mae: 0.1145 - val_loss: 0.1017 - val_mae: 0.1017
Epoch 3/50
10/10 [==============================] - 0s 14ms/step - loss: 0.1003 - mae: 0.1003 - val_loss: 0.0966 - val_mae: 0.0966
Epoch 4/50
10/10 [==============================] - 0s 14ms/step - loss: 0.0963 - mae: 0.0963 - val_loss: 0.0939 - val_mae: 0.0939
Epoch 5/50
10/10 [==============================] - 0s 14ms/step - loss: 0.0937 - mae: 0.0937 - val_loss: 0.0922 - val_mae: 0.0922
Epoch 6/50
10/10 [==============================] - 0s 19ms/step - loss: 0.0925 - mae: 0.0925 - val_loss: 0.0909 - val_mae: 0.0909
Epoch 7/50
10/10 [==============================] - 0s 13ms/step - loss: 0.0912 - mae: 0.0912 - val_loss: 0.0898 - val_mae: 0.0898
Epoch 8/50
10/10 [==============================] - 0s 15ms/step - loss: 0.0900 - m

In [ ]:
X_encoded

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.0,0.000000,0.0,0.0,0.0,0.444481,0.0,1.205548,0.0,3.605999,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.100807,0.0,0.0,0.0,2.907100,0.0,0.352874,0.0,2.428939,...,0.0,0.0,0.0,2.213841,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.411632,0.0,0.858984,0.0,2.668936,...,0.0,0.0,0.0,1.740264,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.028928,0.0,0.0,0.0,1.455846,0.0,0.524559,0.0,2.646515,...,0.0,0.0,0.0,1.341616,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.965520,0.0,1.418398,0.0,2.194422,...,0.0,0.0,0.0,1.634098,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,0.0,0.000000,0.0,0.0,0.0,0.216280,0.0,1.403962,0.0,3.486822,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
446,0.0,0.069891,0.0,0.0,0.0,1.509834,0.0,0.360361,0.0,3.201774,...,0.0,0.0,0.0,0.662389,0.0,0.0,0.0,0.0,0.0,0.0
447,0.0,0.394299,0.0,0.0,0.0,1.208383,0.0,0.934140,0.0,3.969682,...,0.0,0.0,0.0,1.258147,0.0,0.0,0.0,0.0,0.0,0.0
448,0.0,0.000000,0.0,0.0,0.0,1.455774,0.0,1.697695,0.0,3.347207,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
text_descriptions

0      TCGA.05.4384
1      TCGA.05.4390
2      TCGA.05.4396
3      TCGA.05.4405
4      TCGA.05.4410
           ...     
445    TCGA.NJ.A55O
446    TCGA.NJ.A55R
447    TCGA.NJ.A7XG
448    TCGA.O1.A52J
449    TCGA.S2.AA1A
Name: attrib_name, Length: 450, dtype: object

In [ ]:
X_encoded.insert(0, 'attrib_name', text_descriptions)
X_encoded

,attrib_name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,TCGA.05.4384,0.0,0.000000,0.0,0.0,0.0,0.444481,0.0,1.205548,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,TCGA.05.4390,0.0,0.100807,0.0,0.0,0.0,2.907100,0.0,0.352874,0.0,...,0.0,0.0,0.0,2.213841,0.0,0.0,0.0,0.0,0.0,0.0
2,TCGA.05.4396,0.0,0.000000,0.0,0.0,0.0,0.411632,0.0,0.858984,0.0,...,0.0,0.0,0.0,1.740264,0.0,0.0,0.0,0.0,0.0,0.0
3,TCGA.05.4405,0.0,0.028928,0.0,0.0,0.0,1.455846,0.0,0.524559,0.0,...,0.0,0.0,0.0,1.341616,0.0,0.0,0.0,0.0,0.0,0.0
4,TCGA.05.4410,0.0,0.000000,0.0,0.0,0.0,0.965520,0.0,1.418398,0.0,...,0.0,0.0,0.0,1.634098,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,TCGA.NJ.A55O,0.0,0.000000,0.0,0.0,0.0,0.216280,0.0,1.403962,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
446,TCGA.NJ.A55R,0.0,0.069891,0.0,0.0,0.0,1.509834,0.0,0.360361,0.0,...,0.0,0.0,0.0,0.662389,0.0,0.0,0.0,0.0,0.0,0.0
447,TCGA.NJ.A7XG,0.0,0.394299,0.0,0.0,0.0,1.208383,0.0,0.934140,0.0,...,0.0,0.0,0.0,1.258147,0.0,0.0,0.0,0.0,0.0,0.0
448,TCGA.O1.A52J,0.0,0.000000,0.0,0.0,0.0,1.455774,0.0,1.697695,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_encoded.to_csv("drive/MyDrive/5703/TCGA-LUAD/TCGA_LUAD_miRNASeq__HS_miR_128.csv", index=False)